# Project: Stock Data Analysis


This notebook contains the code breakdown for the Stock Data Analysis project, where we analyze and visualize the performance of MAANG stocks against the S&P 500. It includes loading data, cleaning and merging datasets, creating visualizations, and generating insights and Showing the vast difference between the impact of MAANG vs the market/S&P500.


## 1. Loading Data

In [7]:

import pandas as pd

def load_data():
    """ Load MAANG and US Stock Market datasets with robust error handling """
    try:
        df_maang = pd.read_csv("Combined MAANG.csv", thousands=',', decimal='.')
        df_market = pd.read_csv("US Stock Market Data.csv", thousands=',', decimal='.')
        
        print("Datasets loaded successfully!")
        return df_maang, df_market
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        return None, None


## 2. Data Cleaning and Merging

In [8]:

def clean_and_merge_data(df_maang, df_market):
    """ Clean datasets and merge them with robust date handling """
    required_maang_cols = ["Date", "Close"]
    required_market_cols = ["Date", "S&P_500_Price"]
    
    for col in required_maang_cols:
        if col not in df_maang.columns:
            raise ValueError(f"Missing required column {col} in MAANG dataset")
    
    for col in required_market_cols:
        if col not in df_market.columns:
            raise ValueError(f"Missing required column {col} in Market dataset")
    
    df_maang = df_maang.ffill()
    df_market = df_market.ffill()
    
    df_maang["Date"] = pd.to_datetime(df_maang["Date"], infer_datetime_format=True, errors='coerce')
    df_market["Date"] = pd.to_datetime(df_market["Date"], infer_datetime_format=True, errors='coerce')
    
    df_maang = df_maang.dropna(subset=["Date"])
    df_market = df_market.dropna(subset=["Date"])
    
    df_maang["Close"] = pd.to_numeric(df_maang["Close"], errors='coerce')
    df_market["S&P_500_Price"] = pd.to_numeric(df_market["S&P_500_Price"], errors='coerce')
    
    df_maang = df_maang.dropna(subset=["Close"])
    df_market = df_market.dropna(subset=["S&P_500_Price"])
    
    df_maang = df_maang[(df_maang["Date"] >= "2020-01-01") & (df_maang["Date"] <= "2022-12-31")]
    df_market = df_market[(df_market["Date"] >= "2020-01-01") & (df_market["Date"] <= "2022-12-31")]
    
    df_merged = pd.merge(df_maang, df_market, on="Date", how="inner")
    df_merged["MAANG_Market_Diff_Pct"] = ((df_merged["Close"] - df_merged["S&P_500_Price"]) / df_merged["S&P_500_Price"]) * 100
    
    return df_merged


## 3. Creating Visualizations (Matplotlib)

In [9]:

import matplotlib.pyplot as plt

def create_matplotlib_visualizations(df_merged):
    """ Create matplotlib visualizations """
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(df_merged["Date"], df_merged["Close"], label="MAANG Stock")
        plt.plot(df_merged["Date"], df_merged["S&P_500_Price"], label="S&P 500")
        plt.title("MAANG Stock Price vs S&P 500 (2020-2022)")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig("stock_price_comparison.png")
        plt.close()

        plt.figure(figsize=(12, 6))
        plt.plot(df_merged["Date"], df_merged["MAANG_Market_Diff_Pct"])
        plt.title("MAANG vs S&P 500 Percentage Difference (2020-2022)")
        plt.xlabel("Date")
        plt.ylabel("Percentage Difference")
        plt.axhline(y=0, color='r', linestyle='--')
        plt.tight_layout()
        plt.savefig("percentage_difference.png")
        plt.close()

    except Exception as e:
        print(f"Error creating matplotlib visualizations: {e}")


## 4. Creating Interactive Visualizations (Plotly)

In [10]:

import plotly.express as px

def create_plotly_dashboard(df_merged):
    """ Create interactive Plotly dashboard """
    try:
        fig = px.line(df_merged, x="Date", y=["Close", "S&P_500_Price"], 
                      title="MAANG Stock vs S&P 500 Interactive Chart")
        fig.write_html("interactive_stock_chart.html")

        fig_diff = px.area(df_merged, x="Date", y="MAANG_Market_Diff_Pct", 
                           title="MAANG vs S&P 500 Percentage Difference")
        fig_diff.write_html("percentage_difference_chart.html")

    except Exception as e:
        print(f"Error creating Plotly dashboard: {e}")


## 5. Creating Pivot Table

In [11]:

def create_pivot_table(df_merged):
    """ Create and save pivot table """
    try:
        pivot_monthly = df_merged.groupby(pd.Grouper(key="Date", freq="M")).agg({
            "Close": "mean",
            "S&P_500_Price": "mean",
            "MAANG_Market_Diff_Pct": "mean"
        }).reset_index()
        
        pivot_monthly.to_csv("monthly_stock_comparison.csv", index=False)
        print("Pivot table created successfully!")

    except Exception as e:
        print(f"Error creating pivot table: {e}")


## 6. Main Function

In [12]:

def main():
    """ Main function to orchestrate data processing and visualization """
    df_maang, df_market = load_data()
    
    if df_maang is not None and df_market is not None:
        try:
            df_merged = clean_and_merge_data(df_maang, df_market)
            df_merged.to_csv("Processed_MAANG_vs_Market.csv", index=False)
            
            create_matplotlib_visualizations(df_merged)
            create_plotly_dashboard(df_merged)
            create_pivot_table(df_merged)
            
            print("Data processing completed successfully!")
        
        except Exception as e:
            print(f"An error occurred during data processing: {e}")
            print("Please check your data files and ensure they are correctly formatted.")

if __name__ == "__main__":
    main()


Datasets loaded successfully!


C:\Users\Felice\AppData\Local\Temp\ipykernel_19176\1769287097.py:17: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Felice\AppData\Local\Temp\ipykernel_19176\1769287097.py:18: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Pivot table created successfully!
Data processing completed successfully!


C:\Users\Felice\AppData\Local\Temp\ipykernel_19176\907023332.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

